In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt


In [ ]:

def load_pkl_file(file_path):
    with open(file_path, 'rb') as file:
        loaded_object = pickle.load(file)
    return loaded_object



In [ ]:
flavdesc=load_pkl_file('/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/2007-10flav/FlavDesc.pkl')

In [ ]:
flavdesc

In [ ]:
flavval=load_pkl_file('/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/2007-10flav/FlavVal.pkl')

In [ ]:
flavval

In [ ]:
columns = ['Food code', 'Nutrient code', 'Start date', 'End date', 'Nutrient value']


df = pd.DataFrame(flavval, columns=columns)


Food_Nut_conc = df.pivot(index='Food code', columns='Nutrient code', values='Nutrient value')

Food_Nut_conc = Food_Nut_conc.reset_index()
Food_Nut_conc.columns.name = None


print(Food_Nut_conc)

In [ ]:
Food_Nut_conc.columns

In [ ]:
import pandas as pd

def read_csv_to_dataframe(file_path):
    try:
        df = pd.read_csv(file_path)
        return df
    except Exception as e:
        print("An error occurred:", e)
        return None


In [ ]:

csv_file_path = '/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/FNDDS_2009-10_Training_Data.csv'
complete_data_w_nova_classification = read_csv_to_dataframe(csv_file_path)


In [ ]:
complete_data_w_nova_classification

In [ ]:
nova_classified_data = complete_data_w_nova_classification[
    complete_data_w_nova_classification['novaclass'].isin([1, 2, 3, 4])
]

(nova_classified_data)

In [ ]:

columns_to_include = ['Food_code', 'Main_food_description', 'catnumb', 'catname', 'novaclass', 'macroclass']

food_coded_nova_class = nova_classified_data[columns_to_include].copy()




In [ ]:
food_coded_nova_class

In [ ]:
nutrient_conc=read_csv_to_dataframe("/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/nutrient_conc_nova_classified.csv")

In [ ]:
nutrient_conc

In [ ]:
nutrient_flav_conc_nova_classified = pd.merge(nutrient_conc, Food_Nut_conc, left_on='Food code', right_on='Food code', how='inner')


In [ ]:
nutrient_flav_conc_nova_classified

In [ ]:
nutrient_dict = {}

for index, row in flavdesc.iterrows():
    nutrient_code = row['Nutrient code']

    unit = row['Unit']
    nutrient_dict[nutrient_code] = {'description': row['Flavonoid description']}



In [ ]:
for nutrient_code in nutrient_dict:
    nutrient_description = nutrient_dict[nutrient_code]['description']
    nutrient_flav_conc_nova_classified.rename(columns={nutrient_code: nutrient_description}, inplace=True)



In [ ]:
nutrient_flav_conc_nova_classified.columns

In [ ]:
nutrient_flav_conc_nova_classified.to_csv('/home/nalin21478/BTP/ML-food-Processing/Numerical_Textual_ML/Data/102_Nuts', index=False)

In [ ]:
def move_column_to_end(df, column_name):
    column = df.pop(column_name)
    df[column_name] = column
    return df

In [ ]:
nutrient_flav_conc_nova_classified=move_column_to_end(nutrient_flav_conc_nova_classified,'novaclass')

In [ ]:
nutrient_flav_conc_nova_classified=nutrient_flav_conc_nova_classified.drop(columns=['Food code'])

In [ ]:
nutrient_flav_conc_nova_classified

In [ ]:
def check_missing_data(df):
    missing_data = df.isnull().sum().sort_values(ascending=False)
    percent_missing = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([missing_data, percent_missing], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [ ]:
check_missing_data(nutrient_flav_conc_nova_classified)

In [ ]:
nutrient_flav_conc_nova_classified=nutrient_flav_conc_nova_classified.drop(columns=['Main_food_description'])

In [ ]:
corr=nutrient_flav_conc_nova_classified.corr()



plt.figure(figsize=(10*3, 8*3))
sns.heatmap(corr,cmap='coolwarm', fmt=".2f")

plt.title('Correlation Heatmap')
plt.show()


In [ ]:
novaclass=nutrient_flav_conc_nova_classified['novaclass']

In [ ]:
nutrient_flav_conc_nova_classified.to_csv('/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Data/102_Data.csv',index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler




nutrient_flav_conc_nova_classified_copy=nutrient_flav_conc_nova_classified.copy()
nutrient_flav_conc_nova_classified_copy=nutrient_flav_conc_nova_classified_copy.drop(columns=['novaclass'])

scaler = StandardScaler()


scaled_data = scaler.fit_transform(nutrient_flav_conc_nova_classified_copy)



In [ ]:
import joblib
joblib.dump(scaler, '/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Models/scaler_65.pkl')

In [ ]:
scaled_data = pd.DataFrame(scaled_data, columns=nutrient_flav_conc_nova_classified_copy.columns)

In [ ]:
scaled_data['novaclass'] = nutrient_flav_conc_nova_classified['novaclass']

In [ ]:
scaled_data

In [ ]:

from sklearn.manifold import TSNE


def apply_tsne_and_group(scaled_data, novaclass,perplexity, num_components, col, output_folder):


    tsne = TSNE(n_components=num_components, perplexity=perplexity, random_state=0)
    tsne_result = tsne.fit_transform(scaled_data)


    tsne_df = pd.DataFrame(tsne_result, columns=[f'tsne_{i+1}' for i in range(num_components)])
    tsne_df['novaclass'] = novaclass

    # Plot and save the t-SNE visualization
    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=tsne_df, x='tsne_1', y='tsne_2', hue='novaclass', palette='Set1')
    plt.title(f"t-SNE Visualization by novaclass (Perplexity: {perplexity}, Column: {col})")

    # Construct the file path for saving the plot
    plot_filename = os.path.join(output_folder, f"tsne_visualization_perplexity{perplexity}_col{col}.png")

    # Save the plot as an image file
    plt.savefig(plot_filename)

    # Show the plot (optional)
    # plt.show()
    

    return tsne_df




In [ ]:
output_folder="/home/nalin21478/BTP/ML-food-Processing/ML in Food Processing/Graphs/flavsnnuts/TSNE"

In [ ]:
apply_tsne_and_group(scaled_data, novaclass, 5, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*2, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*3, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*4, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*5, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*6, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*7, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*8, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*9, 2, 'novaclass', output_folder)
apply_tsne_and_group(scaled_data, novaclass, 5*10, 2, 'novaclass', output_folder)

In [ ]:
from sklearn.model_selection import train_test_split
Independent_features = scaled_data.iloc[:, :-1].values
dependent_feature = scaled_data.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(
        Independent_features, dependent_feature, test_size=0.2, random_state=0
    )

In [ ]:
dependent_feature

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold

def evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, classifier, num_folds=10):

    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)
    

    accuracies = []


    for train_indices, val_indices in k_fold.split(X_train, y_train):
        X_fold_train, X_fold_val = X_train[train_indices], X_train[val_indices]
        y_fold_train, y_fold_val = y_train[train_indices], y_train[val_indices]

        
        classifier.fit(X_fold_train, y_fold_train)

    
        y_val_pred = classifier.predict(X_fold_val)

     
        fold_accuracy = accuracy_score(y_fold_val, y_val_pred)
        accuracies.append(fold_accuracy)

   
    average_accuracy = sum(accuracies) / num_folds
    print(f'Average Accuracy Train: {average_accuracy*100}')

    
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy Test: {accuracy*100:.2f}%')
    
    
    class_report = classification_report(y_test, y_pred)
    print('Classification Report:')
    print(class_report)
    
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:')
    print(conf_matrix)





In [ ]:
from sklearn.tree import DecisionTreeClassifier
print("DECISION TREE ENTROPY \n")
decision_tree_entropy=DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, decision_tree_entropy, num_folds=10)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
print("DECISION TREE ENTROPY \n")
decision_tree_entropy=DecisionTreeClassifier(criterion = 'gini', random_state = 0)
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, decision_tree_entropy, num_folds=10)

In [ ]:

from sklearn.ensemble import RandomForestClassifier
print("RANDOM FOREST CLASSIFIER \n")
random_forest_classifier = RandomForestClassifier(random_state=42)
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, random_forest_classifier, num_folds=10)




In [ ]:

from sklearn.naive_bayes import GaussianNB
print("NAIVE BAYES CLASSIFIER \n")

# Initialize the Naive Bayes Classifier
nb_classifier = GaussianNB()

evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, nb_classifier, num_folds=10)


In [ ]:

from sklearn.svm import SVC

print("SVM LINEAR CLASSIFIER \n")
# Initialize the SVM Classifier with a linear kernel
svm__linear_classifier = SVC(kernel='linear', random_state=0)

evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, svm__linear_classifier, num_folds=10)

In [ ]:

from sklearn.svm import SVC

print("SVM poly CLASSIFIER \n")
# Initialize the SVM Classifier with a linear kernel
svm__linear_classifier = SVC(kernel='poly', random_state=0)

evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, svm__linear_classifier, num_folds=10)

In [ ]:

from sklearn.svm import SVC

print("SVM RBF CLASSIFIER \n")

svm__rbf_classifier = SVC(random_state=0)
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, svm__rbf_classifier, num_folds=10)

In [ ]:

from sklearn.svm import SVC

print("SVM SIGMOID CLASSIFIER \n")

svm__sigmoid_classifier = SVC(kernel='sigmoid', random_state=0)
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, svm__sigmoid_classifier, num_folds=10)

In [ ]:

from sklearn.neighbors import KNeighborsClassifier



knn_classifier = KNeighborsClassifier()




print("KNN CLASSIFIER \n")
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, knn_classifier, num_folds=10)



In [ ]:

from sklearn.linear_model import LogisticRegression

lr_classifier = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000)

print("LOGISTIC REGRESSION CLASSIFIER \n")
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, lr_classifier, num_folds=10)


In [ ]:

from sklearn.ensemble import GradientBoostingClassifier


# Initialize the Gradient Boosting Classifier
gb_classifier = GradientBoostingClassifier(n_estimators=100, random_state=0)
print("GRADIENT BOOSTING CLASSIFIER \n")
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, gb_classifier, num_folds=10)


In [ ]:

from sklearn.ensemble import AdaBoostClassifier

ab_classifier = AdaBoostClassifier( random_state=0)
print("ADABOOST CLASSIFIER \n")
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, ab_classifier, num_folds=10)



In [ ]:

from sklearn.neural_network import MLPClassifier



mlp_classifier = MLPClassifier( random_state=0)
print("MLP CLASSIFIER \n")
evaluate_classifier_with_kfold(X_train, y_train, X_test, y_test, mlp_classifier, num_folds=10)

